<a href="https://colab.research.google.com/github/paolaserra/Algorithms-for-massive-datasets/blob/main/ADM_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#starting the project

In [ ]:
# upload token file kaggle.json

from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"giuliahadjiandrea","key":"fc0ce37d8f89c0421a66fda0695b7b94"}'}

In [ ]:
# move the token file to the ./kaggle directory

! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

### **Access Kaggle API, download the "Old Newspapers" dataset and unzip it**

In [ ]:
# authenticate to the Kaggle API
!pip install kaggle
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [ ]:
# download and unzip "Old Newspapers" dataset

!kaggle datasets download -d alvations/old-newspapers

100% 2.04G/2.05G [00:19<00:00, 152MB/s]
100% 2.05G/2.05G [00:19<00:00, 112MB/s]


In [ ]:
!unzip old-newspapers.zip

Archive:  old-newspapers.zip
  inflating: old-newspaper.tsv       


### **Initialize a SparkContext object**


In [ ]:
# check number of available cores

!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

In [ ]:
# initialize a SparkContext object 

! pip install pyspark

import pyspark
sc = pyspark.SparkContext()

     |████████████████████████████████| 281.4 MB 25 kB/s 
     |████████████████████████████████| 198 kB 52.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=09b04532e6335d0738329c0bf64fe463cd0b78e5c2dcae1e27b48659b48b3eaa
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
#install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null  


In [ ]:
# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.2/spark-3.0.2-bin-hadoop3.2.tgz


In [ ]:
# unzip the spark file to the current folder
!tar xf spark-3.0.2-bin-hadoop3.2.tgz

#install findspark
!pip install -q findspark

In [ ]:
# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init("spark-3.0.2-bin-hadoop3.2")  #SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

**PREPROCESSING**

Content
The columns of each row in the .tsv file are:

Langauge: Language of the text.

Source: Newspaper from which the text is from.

Date: Date of the article that contains the text.

Text: Sentence/paragraph from the newspaper


In [ ]:
#DATASET to DF
import pandas
df = spark.read.csv("/content/old-newspaper.tsv", sep=r'\t', header=True)

In [ ]:
df.head(1)

[Row(Language='Afrikaans', Source='republikein.com.na', Date='2011/09/14', Text='Die veranderinge aan die Britsgeboude Avensis sluit in hersiene stilering aan die buitekant, wat n voorkantontwerp ingevolge Toyota se nuwe ontwerpstyl insluit.')]

In [ ]:
# create an RDD from the "Old Newspapers" dataset and display the first instance

newsRDD = sc.textFile("old-newspaper.tsv", 2)
newsRDD.take(1)

['Language\tSource\tDate\tText']

In [ ]:
def split_row(x):

  tup = x.split('''\t''')
  
  date = tup[2]
  year = date[:4]  

  return (tup[0], year, tup[3])

In [ ]:
subRDD = newsRDD.map(split_row)
subRDD.take(1)

[('Language', 'Date', 'Text')]

In [ ]:
# count by years

subRDD.map(lambda t: t[1]).countByValue()

defaultdict(int,
            {'1970': 6,
             '2000': 6075,
             '2001': 7811,
             '2002': 6183,
             '2003': 8452,
             '2004': 20091,
             '2005': 77733,
             '2006': 169678,
             '2007': 369179,
             '2008': 687741,
             '2009': 1494895,
             '2010': 3207680,
             '2011': 7247763,
             '2012': 2812367,
             '2013': 380469,
             'Date': 1,
             'UNKN': 309918})

In [ ]:
# look directly at those with year = 'Date' or year = '1970'

subRDD.filter(lambda t: t[1]=='Date' or t[1]=='1970') \
      .collect()

[('Language', 'Date', 'Text'),
 ('French',
  '1970',
  "Habitué depuis deux ans à faire venir des attaquants au mercato hivernal (Quercia en 2008 et Birsa en 2009), Jean Fernandez justifie les arrivées de Jo-Gook et Fall comme des choix « d'avenir ». Et éclaire sur la capacité de ces joueurs à réussir à court terme. « L'expérience me fait dire qu'il y a plus d'échecs que de réussite au mercato d'hiver. Lorsque j'étais à Marseille, Maoulida et Pagis avaient été de bonnes recrues »."),
 ('French',
  '1970',
  "Deux joueurs qui n'ont rien coûté, ou presque, à l'AJA fidèle à sa stratégie de recruter malin. Jo-Gook, auteur d'une apparition en Coupe de France et une en Ligue 1, a confirmé que « le Patriot », comme il est surnommé en Corée du Sud, aura besoin de quelques mois d'adaptation."),
 ('French',
  '1970',
  "Ce que l'on sait, d'ores-et-déjà, c'est que Jelen (en fin de contrat en juin) a peu de chances d'en faire partie. La tendance de l'hiver ne sera peut-être pas celle de l'été, cer

In [ ]:
# look at the language in which the ones with unknown date were written

subRDD.filter(lambda t: t[1]=='UNKN') \
      .map(lambda t: t[0]) \
      .countByValue()  # all in Spanish

defaultdict(int, {'Spanish': 309918})

In [ ]:
textRDD = subRDD.filter(lambda x: x[0]=='French')